In [1]:
library(gganimate)
library(tidyverse)

Loading required package: ggplot2
Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ tibble  2.1.1       ✔ purrr   0.3.2  
✔ tidyr   0.8.3       ✔ dplyr   0.8.0.1
✔ readr   1.3.1       ✔ stringr 1.4.0  
✔ tibble  2.1.1       ✔ forcats 0.4.0  
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter()  masks stats::filter()
✖ purrr::flatten() masks jsonlite::flatten()
✖ dplyr::lag()     masks stats::lag()


In [2]:
Data <- readxl::read_xlsx("FootballData.xlsx")

In [3]:
names(Data) <- c("Club","Income","Country","Year","Position","Colour")

In [4]:
Data$ClubYear <- paste(Data$Club, Data$Year)

In [5]:
Data$First <- ifelse(Data$Position == 1, 1,0)

In [34]:
Data$FirstSum <- NA
for (i in 1:nrow(Data)){
    temp <- Data[1:i,]
    temp <- sum(temp$First[temp$Club==Data$Club[i]], na.rm = TRUE)
    Data$FirstSum[i] <- as.numeric(temp)
}

In [37]:
Data <- Data %>%
    group_by(Year) %>%
    mutate(rank = order(order(FirstSum, Club, decreasing=TRUE)))

[1] "Club"     "Income"   "Country"  "Year"     "Position" "Colour"   "ClubYear"
[8] "First"    "FirstSum"

In [50]:
Data2 <- Data[Data$rank < 10 & Data$FirstSum >0,]

In [51]:
Graphs <- 
ggplot(Data2, aes(rank, group = Club))+
geom_col(aes(y=FirstSum, color = Club, fill = Club))+
coord_flip(clip = "off", expand = FALSE) +
  scale_x_reverse() +
  guides(color = FALSE, fill = FALSE) + #Add Watermark With Website Name
  annotate(
    "text",
    x = -0.5,
    y = 0,
    label = "www.TheDataInspector.com",
    #vjust = -5.5,
      hjust = -1.5,
    colour = "#8D8D8D",
    size = 6
  ) +
  geom_text(
    aes(
      y = 0,
      label = paste(Club, " "),
        color = Club
    ),
    vjust = 0.2,
    hjust = 1,
    size = 6
  ) + 
geom_text(
    aes(
      #y = max(Income) * 0.2,
      y = 0,
        x = -0.5,
      label = as.character(Year),
      fontface = 2,
        color = Club
    ),
    size = 12,
    #vjust = -2.5,
    colour = "#8D8D8D",
    check_overlap = TRUE
  ) +
 geom_text(aes(y = FirstSum, label = as.character(FirstSum)),
            size = 8,
            nudge_y = 10) +
  labs(title = "European Football Club Revenue")+
  theme(
    #Set Theme Options
    axis.line = element_blank(),
    axis.text.y = element_blank(),
    axis.text.x = element_blank(),
    axis.ticks = element_blank(),
    axis.title = element_blank(),
    legend.position = "none",
    panel.background = element_blank(),
    panel.border = element_blank(),
    panel.grid.major = element_blank(),
    panel.grid.minor = element_blank(),
    panel.grid.major.x = element_line(size = .05, colour = "#DBDBDB"),
    panel.grid.minor.x = element_line(size = .05, colour = "#DBDBDB"),
    plot.title = element_text(
      size = 30,
      hjust = 0.5,
      face = "bold",
      colour = "#8D8D8D",
      vjust = 5
    ),
    plot.background = element_blank(),
    plot.margin = margin(3, 2, 2, 10, "cm")
  ) + #Add Transition State Controlling Time Spent Moving And Time Spent At Data Point
  transition_states(Year, transition_length = 10, state_length = 1) + #Control If X & Y Axis Should Scale As Data Changes
  view_follow(fixed_x = TRUE, fixed_y = TRUE)

In [52]:
animate(Graphs, height = 750, width = 900, nframes = 200, fps = 20, renderer = gifski_renderer("Football.gif"))

In [53]:
CountryData <- unique.data.frame(Data[c("Country","First","FirstSum")])